# A1000 YARA rules API
This notebook contains code examples for the A1000 YARA API. This API enables management of our custom YARA rulesets on our A1000 instance. It has several endpoints and actions available for use.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md) 

### 1. Import the A1000 class

In [ ]:
from ReversingLabs.SDK.a1000 import A1000

### 2. Loading the credentials
Next, we will load our A1000 credentials and host URL from the local `a1000_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

In [ ]:
import json


CREDENTIALS = json.load(open("a1000_credentials.json"))
HOST = CREDENTIALS.get("host")
TOKEN = CREDENTIALS.get("token")
USER_AGENT = "ReversingLabs SDK Cookbook v1.0.2"

#### 3. Creating an A1000 object
First let us create an object out of the A1000 class. Unlike the `ticloud` module, the `a1000` module has only one class for all its methods (API-s).
Having that in mind, we will create our A1000 object:

In [ ]:
a1000 = A1000(
    host=HOST,
    token=TOKEN,
    verify=False,
    user_agent=USER_AGENT
)

**NOTE:** Set the `verify` parameter to `True` or `False` depending on whether you want your server certificates to be validated or ignored.

### 4. Creating the ruleset
YARA enables rule-based identification and classification of files. To create a new YARA ruleset on your A1000 instance, do the following steps:

In [ ]:
ruleset_text = """
    rule PdfsWithUrls
    {
    strings:
        $Hex_PDF_magic = { 25 50 44 46 }
        $Regex_URL = /https?:\/\/([\w\.-]+)([\/\w \.-]*)/
    
    condition:
        filesize > 10KB and filesize < 200KB and
        $Hex_PDF_magic at 0 and
        $Regex_URL
    }
"""

response = a1000.create_or_update_yara_ruleset(
    name="PdfsWithUrls",
    content=ruleset_text
)

print(response.text)

Parameters explained:
- `name`: A name for your YARA ruleset.
- `content`: The ruleset itself. Here you must paste the rule text in the form of a string.  

The ruleset that we used will detect all PDF files on A1000 that contain URL-s starting with "https". This ruleset is not particularly useful for detecting malware, but we are using it here for the sake of demonstration, since it is likely for it to work well with a lot of files found on your instance of A1000.

### 5. Enabling the ruleset
After we have successfully created our ruleset, we need to enable it. To do so, follow up with these steps:

In [ ]:
response = a1000.enable_or_disable_yara_ruleset(
    enabled=True,
    name="PdfsWithUrls"
)

print(response.text)

Parameters explained:
- `name`: The name of the ruleset we want to manage.
- `enabled`: A boolean statement whether that ruleset should be enabled or not.

After receiving a response message that confirms us that our ruleset is enabled, we are good to go. However, let us first check what our new ruleset looks like on our A1000.

### 6. Listing the available rulesets
To see the status of the existing rulesets on our A1000, we need to list them out using another A1000 class method.

In [ ]:
response = a1000.get_yara_rulesets_on_the_appliance_v2(
    status="active"
)

print(response.text)

We used "active" as a filter here to return ony existing rulesets that are considered active and running. Since we specifically activated our new ruleset, you should be able to see it in this response. Keep in mind that by not "publishing" the ruleset also to TitaniumCloud, the status of our ruleset should be displayed as `"pending"`. Nevertheless, our ruleset is up and running.

### 7. Getting the ruleset matches
Finally, we can fetch all the matches that our new ruleset has made out of all the samples available on your A1000 instance. Keep in mind that the ruleset that we used here is quite inclusive but there is always a chance that you don't have any PDF files containing URL-s on your A1000.

In [ ]:
response = a1000.get_yara_ruleset_matches_v2(
    ruleset_name="PdfsWithUrls"
)

print(response.text)

If you have any PDF files containing URL-s on your A1000 instance, you should see them listed in the response here. If not, this request will simply succeed without returning any matches.